In [21]:
from py2neo import Node, Relationship, authenticate, Graph

# set up authentication parameters
authenticate("localhost:7474", "neo4j", "neo4j1")

# connect to authenticated graph database
db = Graph("localhost:7474")

# create alice and bob

In [ ]:
"/var/lib/neo4j/import/"
"sudo rm -rf /var/lib/neo4j/data/*"

In [19]:
importActors = '''USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM "file:/actors.csv" AS row
CREATE (:Act {Actor_Id: row.Actor_Id, Name: row.Name});'''

importFilms = '''USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM "file:/film.csv" AS row
CREATE (:Film {
DVD_Title:row.DVD_Title, 
Studio: row.Studio,
Released: row.Released,
Status: row.Status,
Sound: row.Sound,
Version: row.Version,
Price: toInt(row.Price),
Rating: toInt(row.Rating),
Year: toInt(row.Year),
Genre: row.Genre,
Aspect: row.Aspect,
UPC: row.UPC,
DVD_Release: row.DVD_Release,
Film_Id: row.Film_Id,
Timestamp: row.Timestamp});'''

importDirectors = '''USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM "file:/directors.csv" AS row
CREATE (:Director {Director_Id: row.Director_Id, Name: row.Name});'''

In [22]:
db.run(importActors)
db.run("""CREATE INDEX ON :Act(Actor_Id)""")

In [23]:
db.run(importFilms)
db.run("""CREATE INDEX ON :Film(Film_Id)""")

In [25]:
db.run("""MATCH (n:Film)
WITH split(n.DVD_Release, '/') as splitted, n
SET n += {Month: toInt(splitted[0]), Day: toInt(splitted[1]), Year: toInt(splitted[2])}
SET n.DVD_Release = splitted[2]+splitted[0]+splitted[1]""")


In [26]:
db.run(importDirectors)
db.run("""CREATE INDEX ON :Director(Director_Id)""")

In [27]:
importA2F = """ 
USING PERIODIC COMMIT 50000
LOAD CSV WITH HEADERS FROM "file:/actor_to_film.csv" AS row
MATCH (a:Act {Actor_Id: row.Actor_Id}), (f:Film {Film_Id: row.Film_Id})
CREATE (a)-[:ACTED]->(f)"""

importD2F = """ 
USING PERIODIC COMMIT 50000
LOAD CSV WITH HEADERS FROM "file:/director_to_film.csv" AS row
MATCH (d:Director {Director_Id: row.Director_Id}), (f:Film {Film_Id: row.Film_Id})
CREATE (d)-[:DIRECTED]->(f)"""

In [28]:
db.run(importA2F)
db.run(importD2F)

# QUERIES

In [ ]:
# QUERY 1 #
## Info on a specific Actor works with a budget of 30$
USE dm_hw1;
SELECT F.DVD_Title, F.Studio, F.Sound, F.Version, F.Genre, F.Year, F.Price, F.UPC, F.Status
FROM (SELECT df.DVD_id
		FROM actor_to_film AS df JOIN actors AS d ON df.Actor_id = d.Actor_id
		WHERE d.Name LIKE 'Jackson Samuel%') as D JOIN film F ON D.DVD_id = F.ID
WHERE F.Price < 30;

In [ ]:
q1 = """PROFILE MATCH (a:Act)-[:ACTED]-(f)
WHERE 	f.Price > 30
RETURN DISTINCT a LIMIT 20"""

In [ ]:
# QUERY 2 #
## All DVDs released during summer
USE dm_hw1;
SELECT DVD_Title, DVD_Release, Genre
FROM film
WHERE DVD_Release BETWEEN '6/1/' AND '9/31/'
ORDER BY DVD_Release;

In [ ]:
q2 = """MATCH (f:Film)
WHERE f.Month >= 6 AND f.Month <= 9
RETURN f.DVD_Title, f.Year, f.Month, f.Day, f.Genre
ORDER BY f.DVD_Release """

In [ ]:
# QUERY 3 #
## Number, greater than 10, of different Directors for each actor
USE dm_hw1;
SELECT DA.Actor_id, A.Name, DA.Count
FROM (SELECT af.Actor_id, COUNT(*) as Count
	  FROM director_to_film AS df JOIN actor_to_film AS af ON df.DVD_id = af.DVD_id
      GROUP BY af.Actor_id) as DA, actors as A
WHERE DA.Actor_id = A.Actor_id
HAVING DA.Count > 10
ORDER BY DA.Count DESC;

In [ ]:
q3 = """ 
MATCH p=(a)-[:ACTED]-(f)-[:DIRECTED]-(d)
WITH COUNT(a) as Collaborations, a 
WHERE Collaborations > 100
RETURN a.Name, Collaborations  
ORDER BY Collaboration DESC"""

In [ ]:
# QUERY 4 #
## All the DVDs that have an actor named Lee or similar

USE dm_hw1;
SELECT F.DVD_Title, F.Price, F.Genre, F.Version as Format
FROM film as F 
WHERE F.ID IN (SELECT D.DVD_id
			     FROM actor_to_film as D
                 WHERE D.Actor_id IN (SELECT A.Actor_id
									  FROM actors as A
                                      WHERE A.Name LIKE 'Lee%'));

In [ ]:
q4 = """MATCH (a:Act)
WHERE a.Name CONTAINS "Lee"
MATCH (a)-[:ACTED]->(f) 
RETURN f.DVD_Title, f.Price, f.Genre, f.Version """

In [ ]:
# QUERY 5 #
# W/O VIEW #
## Out of productions DVD on 50% sale
USE dm_hw1;
SELECT DVD_Title, Price, Status, ROUND(0.5*Price, 2) AS Sales
FROM film
WHERE Status = 'Discontinued';


In [ ]:
q5 = """MATCH (f:Film {Status : "Discontinued"})
RETURN f.DVD_Title, f.Price, f.Status, ROUND(0.5*f.Price) AS Sales """

In [ ]:
# QUERY 6 #
# W/O VIEW & REFORMULATION
## All the DVDs that have an actor named Lee or similar 
SELECT F.DVD_Title, F.Price, F.Genre, F.Version as Format
FROM film as F 
WHERE F.ID IN (SELECT D.DVD_id
			     FROM actor_to_film as D
                 WHERE D.Actor_id IN (SELECT A.Actor_id
									  FROM actors as A
                                      WHERE A.Name LIKE 'Lee%'));
                                      
# WITH VIEW & REFORMULATION
## All the DVDs that have an actor named Lee or similar
SELECT F.DVD_Title, F.Price, F.Genre, F.Version as Format
FROM film as F, Lee
WHERE F.ID IN (Lee.DVD_id);

In [ ]:
q6 = """MATCH (a:Act)
WHERE a.Name CONTAINS "Lee"
MATCH (a)-[:ACTED]->(f) 
RETURN f.DVD_Title, f.Price, f.Genre, f.Version  """